In [2]:
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
from future.utils import iteritems
from collections import Counter
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pymysql
from gensim.models import word2vec
from tqdm.notebook import tqdm

import sys
if sys.version_info <= (2,7):
    reload(sys)
    sys.setdefaultencoding('utf-8')
import konlpy
from konlpy.tag import Kkma, Okt, Hannanum

In [56]:
article_data = pd.read_csv('mixver9.txt', encoding='utf-8', header= None)
documents = [' '.join(i[0].split(' ')[1:]) for i in article_data.values]

as_one = ''
for document in documents:
    as_one = as_one + ' ' + document
words = as_one.split()

counts = Counter(words)

vocab = sorted(counts, key=counts.get, reverse=True)

word2idx = {word.encode("utf8").decode("utf8"): ii for ii, word in enumerate(vocab,1)}

idx2word = {ii: word for ii, word in enumerate(vocab)}

V = len(word2idx)
N = len(documents)

tf = CountVectorizer()

tf.fit_transform(documents)

tf.fit_transform(documents)[0:1].toarray()

tfidf = TfidfVectorizer(max_features = 1800, max_df=1, min_df=0)

#generate tf-idf term-document matrix
A_tfidf_sp = tfidf.fit_transform(documents)  #size D x V

tfidf_dict = tfidf.get_feature_names()

In [4]:
kkma = Kkma()
okt = Okt()
hannanum = Hannanum()
print('konlpy version = %s' % konlpy.__version__)

konlpy version = 0.5.2


In [18]:
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password='0000',db='test', charset='utf8')
curs = conn.cursor()

project_title = '꿈을 담다. 꿈 해몽 카드 모르몽'
project_category ='디자인'

tokenized_project_title = set(okt.nouns(project_title))
selected_word=[]

for i in tokenized_project_title:
    if i in tfidf_dict:
        selected_word.append(i)
        continue
model = word2vec.Word2Vec.load("NaverMovie29.model")

print(selected_word)

t_cnt =0
w_cnt = 0

for i in selected_word:
    similar_word=model.wv.most_similar(positive=[i],topn=5)
    #print(similar_word)
    for j in similar_word:
        sql = 'select pagename,title from test.crawl where category="%s" and title like "%%%s%%" and achieve>=100;'%(project_category,j[0])
        curs.execute(sql)
        pagename = curs.fetchall()
        #print("pagename")
        #print(type(pagename))
        print(j)
        for k in pagename:
            if k[0] == 'tumblbug':
                print("T "+k[1])
                t_cnt+=1
            elif k[0] == 'wadiz':
                print("W "+k[1])
                w_cnt+=1
            else:
                continue
        print()
print()
print("T W")
print(t_cnt, w_cnt)
if t_cnt > w_cnt:
    print("tumblbug")
elif t_cnt == w_cnt:
    print("same")
else:
    print("wadiz")
conn.close()

['카드']
('지갑', 0.6550707817077637)
T 한국의 울림을 전하다 한울 카드지갑
T 버려졌어요 다시 사랑받고 싶어요. 변치 않을 노트지갑
T 메세지를 담은 나만의 지갑 구성하기 하프문 지갑
T 카드지갑에 쏙  일상감성 손거울 나갈새 이쁘새 행복할새
T 클래식 감성 데일리 지갑 겸 파우치

('동전', 0.6082203388214111)

('지폐', 0.6006042957305908)

('사두', 0.5309817790985107)
W 사두면 1년이 든든한 카드 종합 세트

('트럼프', 0.5300806760787964)


T W
5 1
tumblbug


In [34]:
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password='0000',db='test', charset='utf8')
curs = conn.cursor()

project_title = '스케치업 웹툰 배경 단독주택가'
project_category ='만화'

tokenized_project_title = set(okt.nouns(project_title))
selected_word=[]

for i in tokenized_project_title:
    if i in tfidf_dict:
        selected_word.append(i)
        continue
model = word2vec.Word2Vec.load("NaverMovie29.model")

print(selected_word)

t_cnt =0
w_cnt = 0

for i in selected_word:
    similar_word=model.wv.most_similar(positive=[i],topn=5)
    #print(similar_word)
    for j in similar_word:
        sql = 'select pagename,title from test.crawl where category="%s" and title like "%%%s%%" and achieve>=100;'%(project_category,j[0])
        curs.execute(sql)
        pagename = curs.fetchall()
        #print("pagename")
        #print(type(pagename))
        print(j)
        print(pagename)
        for k in pagename:
            if k[0] == 'tumblbug':
                print("T "+k[1])
                t_cnt+=1
            elif k[0] == 'wadiz':
                print("W "+k[1])
                w_cnt+=1
            else:
                continue
        print()
print()
print("T W")
print(t_cnt, w_cnt)
if t_cnt > w_cnt:
    print("tumblbug")
elif t_cnt == w_cnt:
    print("same")
else:
    print("wadiz")
conn.close()

['웹툰', '스케치업', '배경']
('스케치업', 0.8010225296020508)
(('tumblbug', '스케치업 웹툰 배경 19세기 사용인의 공간 9종세트'), ('tumblbug', '스케치업 해안마을과 소방차 이삿짐차'), ('tumblbug', '스케치업 웹툰배경 귀족클럽과 집무실'), ('tumblbug', '스케치업 웹툰배경 무협 사극 동양풍 배경'), ('tumblbug', '스케치업 웹툰 배경 신비로운 숲 속 마을'), ('tumblbug', '웹툰스케치업배경동양환타지무협사극문파'), ('tumblbug', '스케치업 웹툰배경 방송국 세트장'), ('tumblbug', '스케치업 웹툰_데이트 코스 18종'), ('tumblbug', '스케치업 웹툰배경 팀프리맨의 무협사극 배경4탄'), ('tumblbug', '투닥토닥 판타지 스케치업 프로젝트 No.19_ 사막마을'), ('tumblbug', '스케치업웹툰배경복합거리+지하철플렛폼빙봉들'), ('tumblbug', '스케치업배경 왕궁침실과 물위의정원 set'), ('tumblbug', '스케치업 웹툰 배경 호텔'), ('tumblbug', '스케치업 로판 일러스트를 위한 평민들의 집'), ('tumblbug', '스케치업 웹툰 배경  빈티지 호텔'), ('tumblbug', 'REDBOX 스케치업 웹툰 배경 공사장 전철 지하도'), ('tumblbug', '세렌 웹툰 스케치업8 호텔 내부세트와 대형서점'), ('tumblbug', '스케치업 배경  조선시대 성곽 도시'), ('tumblbug', '투닥토닥 판타지 스케치업 프로젝트 No.17_ 휴식'), ('tumblbug', '스케치업 웹툰 배경 로판 섬 왕국 패키지'), ('tumblbug', '스케치업 웹툰 배경 펜트하우스'), ('tumblbug', '스케치업 웹툰  정원과 수영장이 있는 주택 펜트하우스'), ('tumblbug', '웹툰 스케치업  테라스 펜트하우스와 자동차'), ('tumblbug', '스케치업한번쯤 나올만한 소

('배경', 0.8292587995529175)
(('tumblbug', '스케치업 웹툰 배경 19세기 사용인의 공간 9종세트'), ('tumblbug', '스케치업 웹툰배경 귀족클럽과 집무실'), ('tumblbug', '스케치업 웹툰배경 무협 사극 동양풍 배경'), ('tumblbug', '스케치업 웹툰 배경 신비로운 숲 속 마을'), ('tumblbug', '스냅툰 웹툰 배경 3종세트'), ('tumblbug', '웹툰스케치업배경동양환타지무협사극문파'), ('tumblbug', '스케치업 웹툰배경 방송국 세트장'), ('tumblbug', '스케치업 웹툰배경 팀프리맨의 무협사극 배경4탄'), ('tumblbug', '스케치업웹툰배경복합거리+지하철플렛폼빙봉들'), ('tumblbug', '스케치업배경 왕궁침실과 물위의정원 set'), ('tumblbug', '스케치업 웹툰 배경 호텔'), ('tumblbug', '고퀄리티 2D배경 No.2 사계절 나무풀 산 전경'), ('tumblbug', '스케치업 웹툰 배경  빈티지 호텔'), ('tumblbug', 'REDBOX 스케치업 웹툰 배경 공사장 전철 지하도'), ('tumblbug', '스케치업 배경  조선시대 성곽 도시'), ('tumblbug', '스케치업 웹툰 배경 로판 섬 왕국 패키지'), ('tumblbug', '고퀄리티 2D 배경 소스 210종 하늘나무돌풀'), ('tumblbug', '스케치업 웹툰 배경 펜트하우스'), ('tumblbug', '웹툰스케치업 배경 서양식 대저택'), ('tumblbug', '웹툰 배경 스케치업_로판 대저택과 비밀의 화원 세트'), ('tumblbug', '스케치업 웹툰배경  로판 공작님의 성'), ('tumblbug', '스케치업 웹툰배경 동동이 워터파크 패키지'), ('tumblbug', 'DOOGY의 웹툰배경 야경이 멋진 도시 홍콩'), ('tumblbug', '스케치업 웹툰 배경  서재 5종 Set'), ('tumblbug', '스케치업 웹툰배경 명품브랜드샵

T 스케치업 로판 일러스트를 위한 평민들의 집
T 스케치업 웹툰 배경  빈티지 호텔
T REDBOX 스케치업 웹툰 배경 공사장 전철 지하도
T 세렌 웹툰 스케치업8 호텔 내부세트와 대형서점
T 스케치업 배경  조선시대 성곽 도시
T 투닥토닥 판타지 스케치업 프로젝트 No.17_ 휴식
T 스케치업 웹툰 배경 로판 섬 왕국 패키지
T 스케치업 웹툰 배경 펜트하우스
T 스케치업 웹툰  정원과 수영장이 있는 주택 펜트하우스
T 웹툰 스케치업  테라스 펜트하우스와 자동차
T 스케치업한번쯤 나올만한 소재 페허도시
T 웹툰스케치업 배경 서양식 대저택
T 웹툰을 위한 스케치업 프로젝트 회사 및 매장
T 웹툰스케치업 숲속마녀와 왕궁마법사의 방
T 스케치업 웹툰 사극 프로젝트  읍성편
T 웹툰 배경 스케치업_로판 대저택과 비밀의 화원 세트
T 스케치업 웹툰배경  로판 공작님의 성
T 스케치업 웹툰_음식점 23종과 음식점 거리
T 스케치업 웹툰배경 동동이 워터파크 패키지
T 웹툰 스케치업 프로젝트 청계천
T 스케치업 웹툰 배경  서재 5종 Set
T 웹툰스케치업 군주의 방_서양식침실
T GOODBOY3 스케치업 사무실
T 스케치업 웹툰배경 명품브랜드샵
T 스케치업 웹툰 배경  상가주택베이커리 카페
T ［스케치업 웹툰배경］아파트빌라 주거공간 배경
T 웹툰스케치업서양식_침실 티 룸
T 웹툰판타지 기숙학원_스케치업
T 스케치업 웹툰배경 컨셉 Vol . 1 비밀기관
T 스케치업 웹툰배경 자연과 함께 하는 주택가
T 세렌 웹툰 스케치업7 테헤란로 도시거리상점가
T 스케치업 사장실 10종류.
T 웹툰 배경 프로젝트 01.주거공간 모음 스케치업
T 스케치업 동양풍 웹툰 배경 황궁
T 웹툰 스케치업 배경뒷골목UP
T 투닥토닥 판타지 스케치업 프로젝트 No.11_로판 성
T 마나캣의 밤거리 웹툰 만화 배경 스케치업
T 일레븐 스케치업 05  한옥주택 한옥마을
T 스케치업 웹툰배경 무협 사극 판타지배경
T 스케치업 공작실_3 절벽과 항구가 만난 판타지 마을
T 웹툰 스케치업배경 정원이 있는 전원주

In [58]:
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password='0000',db='test', charset='utf8')
curs = conn.cursor()

project_title = '2년 연속 김치품평회 장관상 100 우리 농산물로 만든 인생김치'
project_category ='푸드'

tokenized_project_title = set(okt.nouns(project_title))
selected_word=[]

for i in tokenized_project_title:
    if i in tfidf_dict:
        selected_word.append(i)
        continue
model = word2vec.Word2Vec.load("NaverMovie29.model")

print(selected_word)

t_cnt =0
w_cnt = 0

similar_word_list=[]
#similar_word_set = set()
for i in selected_word:
    similar_word=model.wv.most_similar(positive=[i],topn=3)
    #print(similar_word)
    for j in similar_word:
#        sql = 'select pagename,title from test.crawl where category="%s" and title like "%%%s%%" and achieve>=100;'%(project_category,j[0])
#        curs.execute(sql)
#        pagename = curs.fetchall()
        #print("pagename")
        #print(type(pagename))
        #print("similar word: ",end='')
        #print(j)
#        print("pagename: ", end='')
#        print(pagename)
        if len(j[0]) == 1:
            print()
            continue
        else:
            similar_word_list.append(j[0])
            #print(len(j[0]))
#        for k in pagename:
#            similar_word_list.append((j[0],k[1]))
        
    
similar_word_set = set(similar_word_list)    
print("similar_word_set",end='')
print(similar_word_set)

res_list=[]

for i in similar_word_set:
    sql = 'select pagename,trim(title) from test.crawl where category="%s" and title like "%%%s%%" and achieve>=100;'%(project_category,i)
    curs.execute(sql)
    pagename = curs.fetchall()
    #print("pagename: ",end='')
    #print(pagename)
    for j in pagename:
        res_list.append((j[0], j[1]))

"""
플젝 명, 카테고리

와디즈랑 텀블벅에 플젝명 토크나이징 -> tfidf -> 검색
개수가져옴
비교
"""
        
        
#print("res list")
#print(res_list)
res_set = set(res_list)
for k in res_set:
    if k[0] == 'tumblbug':
        print("T "+k[1])
        t_cnt+=1
    elif k[0] == 'wadiz':
        print("W "+k[1])
        w_cnt+=1
    else:
        continue
    print()
print()
print("T W")
print(t_cnt, w_cnt)
if t_cnt > w_cnt:
    print("tumblbug")
elif t_cnt == w_cnt:
    print("same")
else:
    print("wadiz")
conn.close()

['김치', '인생']
similar_word_set{'배추', '주인공', '후반', '두부', '엄선', '남현'}
W 4.7점 앵콜김치품평회 대상 국민 김치와 비건이 만나다 남현김치

W 보다 부드럽고 불편함 없이 쉽게 마시는 양배추 양파호박 비트즙!

W 김치품평회 대상 34년 장인의 비법을 담은 내손내만 김장키트남현김치

W 업글앵콜과자끊기힘들죠?갈아타세요!덜단덜짠 슴슴한 두부과자두부43

W 북한국민음식 건강한 곤드레 두부밥과 이북식 만능 양념장

W 72시간 한정 월 9천원대 l 엄선된 원료 정직한 가격의 인생유산균

W 인생유산균 월 9천원대 엄선된 원료 정직한 가격의 프로바이오틱스

W 배추 총각 깍두기 백김치까지 맛있는 전라도김치가 왔다!

W 국산콩두부가 55들어간 수제 비건과자 모르는 사람있나? 탕! 또 있나?

W 업템포X바디프로필 2021년 인생몸매 주인공을 찾습니다 I 퓨어워터


T W
0 10
wadiz


In [59]:
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password='0000',db='test', charset='utf8')
curs = conn.cursor()

project_title = ' 역대급 퀄러티와 가격 프리미엄 캐시미어100 니트머플러'
project_category ='패션'

tokenized_project_title = set(okt.nouns(project_title))
selected_word=[]

for i in tokenized_project_title:
    if i in tfidf_dict:
        selected_word.append(i)
        continue
model = word2vec.Word2Vec.load("NaverMovie29.model")

print(selected_word)

t_cnt =0
w_cnt = 0

similar_word_list=[]
#similar_word_set = set()
for i in selected_word:
    similar_word=model.wv.most_similar(positive=[i],topn=3)
    #print(similar_word)
    for j in similar_word:
#        sql = 'select pagename,title from test.crawl where category="%s" and title like "%%%s%%" and achieve>=100;'%(project_category,j[0])
#        curs.execute(sql)
#        pagename = curs.fetchall()
        #print("pagename")
        #print(type(pagename))
        #print("similar word: ",end='')
        #print(j)
#        print("pagename: ", end='')
#        print(pagename)
        if len(j[0]) == 1:
            print()
            continue
        else:
            similar_word_list.append(j[0])
            #print(len(j[0]))
#        for k in pagename:
#            similar_word_list.append((j[0],k[1]))
        
    
similar_word_set = set(similar_word_list)    
print("similar_word_set",end='')
print(similar_word_set)

res_list=[]

for i in similar_word_set:
    sql = 'select pagename,trim(title) from test.crawl where category="%s" and title like "%%%s%%" and achieve>=100;'%(project_category,i)
    curs.execute(sql)
    pagename = curs.fetchall()
    #print("pagename: ",end='')
    #print(pagename)
    for j in pagename:
        res_list.append((j[0], j[1]))

"""
플젝 명, 카테고리

와디즈랑 텀블벅에 플젝명 토크나이징 -> tfidf -> 검색
개수가져옴
비교
"""
        
        
#print("res list")
#print(res_list)
res_set = set(res_list)
for k in res_set:
    if k[0] == 'tumblbug':
        print("T "+k[1])
        t_cnt+=1
    elif k[0] == 'wadiz':
        print("W "+k[1])
        w_cnt+=1
    else:
        continue
    print()
print()
print("T W")
print(t_cnt, w_cnt)
if t_cnt > w_cnt:
    print("tumblbug")
elif t_cnt == w_cnt:
    print("same")
else:
    print("wadiz")
conn.close()

['니트', '프리미엄', '캐시미어', '가격', '머플러']

similar_word_set{'체스터필드', '나일론', '밍크', '메인', '서브', '바오밥', '미군', '핫팩', '파김치', '캐시', '제일모직', '황금비율', '차이', '얼그레이'}
W 휘뚜루마뚜루 올겨울 여성들을 책임질 9만원대 울캐시미어 4 니트 셋업

W 머플러에 핫팩이 들어간다고? 100 신슐레이트 패딩 머플러4가지컬러

W 국민니트캐시미어 10 황금비율 매일 입기 좋은 니트가 3만원 대!

W 야크울을 아시나요?캐시미어만큼 따뜻하고 더 튼튼한 야크울 니트

W 앵콜메리노울과 미군 방한 패딩의 역대급 조합의 탄생 울프리마 코트

W 어서와~ 캐시미어100은 처음이지?

W 20만원대로 만나는 이탈리아 캐시미어로 만든 핸드메이드 프리미엄 코트

W 가격파괴거품은 0 캐시미어는 100작정하고 만든 뉴노멀 캐시미어

W 코트의 끝 38년 장인이 만든 완벽한 캐시미어 핸드메이드 코트

W 캐시미어100 2탄가장 반응 좋았던 4가지 컬러 머플러만! 단독 펀딩

W 평점5.0앵콜40년 장인이 만든 캐시미어 울 코트 1 클래식코트

W 클래식 코트의 정점 캐시미어로 만든 왕실도 사랑한 폴로코트노블레스

W 격식이 느껴지는 클래식의 정석. 100 캐시미어로 만든 머플러

W 인생코트 캐시미어머플러 선물! 프리미엄 메리노울과 기능성 패딩의 만남

W 100 Real 캐시미어 99000원 진짜 캐시미어가 나타났다

W 디자이너의 손길로 한 치수 날씬하게 보이는 홀가먼트 캐시미어울 스웨터

W 50년 전통 100 프리미엄 캐시미어 컬렉션! 니트부터 머플러까지

W 앵콜 2만원대 홀가먼트 캐시미어 반목 니트

W 캐시미어 니트는 너무비싸 그래서 만든 저렴하지만 좋은 캐시페이크 니트

W 앵콜펀딩유명 브랜드들만 사용하는 프리미엄 양면 캐시미어100 머플러

W ITALI로로피아나 하이엔드 명품소재우린 질적으로달라캐시미어폴로코트

W 앵콜1369달성 올해마지막 캐시미어100 남성코트 100장한